In [ ]:
# George Soros content sources
yt_interview_links = [
    "https://www.youtube.com/watch?v=VP77N0p3cBA&t=7s&ab_channel=ManufacturingIntellect",
    "https://www.youtube.com/watch?v=A0WvKEbq8v0&ab_channel=ManufacturingIntellect",
    "https://www.youtube.com/watch?v=RHSEEJDKJho&ab_channel=OpenSocietyFoundations",
    "https://www.youtube.com/watch?v=ZXyu8u_hsIA&ab_channel=BloombergTelevision",
    "https://www.youtube.com/watch?v=_ONOAJYwaEY&ab_channel=WorldEconomicForum"
    "https://www.youtube.com/watch?v=gm6Bh51kQqc&ab_channel=ColumbiaSIPA"
]

books = {
    ("The Alchemy of Finance") : "/content/the_alchemy_of_finance.pdf",
    ("The Crisis of Global Capitalism") : "/content/the_crisis_of_global_capitalism-chap-5-2017_10_05.pdf"
}

In [ ]:
%pip install youtube-transcript-api

In [2]:
%pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
%pip install --no-deps packaging ninja einops flash-attn trl peft accelerate bitsandbytes
%pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Using cached ninja-1.11.1.2-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (422 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.5 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.7.2.post1-cp310-cp310-linux_x86_64.whl size=190160474 sha256=0b454d9e650bfc437cc71335080172a5d05f51eab355636c9d5b7321fec7318e
  Stored in directory: /root/.cache/pip/wheels/da/ec/5b/b2c37a8e4f755ad82492a822463bca0817f0e0e11de874b550
Successfully built flash-attn
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-4ifpoi0l/unsloth_5f023ed9db164e7faee07724a169d57d
  Running command git clone --filter=blob:none --

In [1]:
import torch
import os
import json
import pandas as pd
from datasets import Dataset, DatasetDict
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Training the model

In [2]:
notebook_login()

In [3]:
from google.colab import files
uploaded = files.upload()  # Upload your JSONL file


Saving data.jsonl to data.jsonl


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('json', data_files='youtube_interview_video.jsonl')

# Load model directly
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")


#3.1

# Load model directly

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
def format_data(examples):
    # Format data into question-answer format for training
    inputs = [f"Question: {q}\nAnswer: {a}" for q, a in zip(examples["question"], examples["answer"])]
    tokenized_inputs = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")

    # Add labels (set them to be the same as input_ids)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].clone()
    return tokenized_inputs

# Set padding token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset with labels
tokenized_dataset = dataset.map(format_data, batched=True)


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, get_peft_model

# Configure LoRA
lora_config = LoraConfig(
    r=16,          # Low-rank dimension
    lora_alpha=32, # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Layers to apply LoRA
    lora_dropout=0.05
)

# Apply LoRA to the model
peft_model = get_peft_model(model, lora_config)

NameError: name 'model' is not defined

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",            # Output directory for checkpoints
    evaluation_strategy="epoch",       # Evaluate every epoch
    learning_rate=2e-5,                # Learning rate
    per_device_train_batch_size=1,     # Batch size per device
    per_device_eval_batch_size=1,      # Batch size for evaluation
    num_train_epochs=10,                # Number of epochs
    logging_dir="./logs",              # Directory to save logs
    logging_steps=10,                  # Log every 10 steps
    save_steps=500,                    # Save model checkpoint every 500 steps
    fp16=True,                          # Use mixed precision for faster training (if supported)
)

# Create the Trainer instance
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"] if "test" in tokenized_dataset else tokenized_dataset["train"],
)

# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.175900,No log
2,2.061000,No log
3,1.988400,No log
4,1.838100,No log
5,1.726100,No log
6,1.731500,No log
7,1.695800,No log
8,1.686400,No log
9,1.598400,No log
10,1.692500,No log


TrainOutput(global_step=200, training_loss=1.8499086570739747, metrics={'train_runtime': 38.1616, 'train_samples_per_second': 5.241, 'train_steps_per_second': 5.241, 'total_flos': 85396864204800.0, 'train_loss': 1.8499086570739747, 'epoch': 10.0})

In [ ]:
%pip install torch

In [ ]:
import torch
# Move the model to the GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
peft_model.to(device)

# Sample question to test the model
test_question = "who are you?"

# Updated prompt format to align with training style
test_input = (
    f"As George Soros, answer the following question in your typical thoughtful and reflective tone:\n"
    f"Question: {test_question}\nAnswer:"
)

# Tokenize the input and move it to the correct device
inputs = tokenizer(test_input, return_tensors="pt").to(device)

# Ensure `pad_token_id` is set
pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id

# Generate an answer
outputs = peft_model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=100,             # Limit the length of the generated response
    num_return_sequences=1,     # Generate a single response
    no_repeat_ngram_size=2,     # Prevent repetition in output
    top_p=0.92,                 # Nucleus sampling (top-p sampling)
    temperature=0.7,            # Control randomness; lower values make output more deterministic
    pad_token_id=pad_token_id   # Explicitly set pad token id
)

# Decode and print the output
generated_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Answer:", generated_answer)


NameError: name 'peft_model' is not defined